In [ ]:
!pip install kaggle
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

from google.colab import drive
drive.mount('./gdrive')

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes
Mounted at ./gdrive


In [ ]:
!pip install -q pytorch-lightning & pip install -q -U albumentations & pip install -q iterative-stratification
!pip install -q timm & pip install -q einops & pip install -q pytorch-lightning wandb & pip install torch-ema
!git clone https://github.com/mlpc-ucsd/CoaT

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.8/225.8 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.4/291.4 kB 22.8 MB/s eta 0:00:00
Cloning into 'CoaT'...
remote: Enumerating objects: 1339, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1339 (delta 10), reused 8 (delta 8), pack-reused 1325 (from 1)
Receiving objects: 100% (1339/1339), 7.10 MiB | 22.66 MiB/s, done.
Resolving deltas: 100% (464/464), done.


In [ ]:
!pip install -q pydicom==2.4.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.9 MB/s eta 0:00:00


In [ ]:
!kaggle competitions download -c rsna-2024-lumbar-spine-degenerative-classification
!unzip -q rsna-2024-lumbar-spine-degenerative-classification.zip

100% 28.2G/28.2G [25:38<00:00, 23.9MB/s]
100% 28.2G/28.2G [25:38<00:00, 19.7MB/s]


In [ ]:
!kaggle datasets download -d wadakoki/rsna-newmeta
!unzip -q /content/rsna-newmeta.zip

Dataset URL: https://www.kaggle.com/datasets/wadakoki/rsna-newmeta
License(s): unknown
100% 3.76M/3.76M [00:01<00:00, 4.15MB/s]
100% 3.76M/3.76M [00:01<00:00, 3.04MB/s]


In [ ]:
import wandb
from pytorch_lightning.loggers import WandbLogger
import os
import yaml
import sys
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from glob import glob
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW, Adam
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR, ReduceLROnPlateau
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
import torchvision.transforms as T
import albumentations as A
import pandas.api.types
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold, StratifiedGroupKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
#from torchvision.models.maxvit import MaxVit
from timm.models.maxxvit import MaxxVit
import timm
import scipy
import albumentations as A
from torchvision.transforms import v2
from torchvision import models
from einops import repeat
from einops.layers.torch import Rearrange
from tqdm.auto import tqdm
#sys.path.append('/content/CoaT')
#from CoaT.src.models.coat import coat_lite_medium
from joblib import Parallel, delayed
from torch.utils.data import default_collate
from torch_ema import ExponentialMovingAverage
import pydicom as dcm
import transformers

## training

In [ ]:
SEED = 126 # friend's birthday
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True # Fix the network according to random seed
    print('Finish seeding with seed {}'.format(seed))

seed_everything(SEED)
print('Training on device {}'.format(device))

Finish seeding with seed 126
Training on device cuda


In [ ]:
%%writefile config.yaml

data_path: "/content/"
output_dir: "/content/gdrive/MyDrive/RSNA_SPINE/models/"

seed: 1101

train_bs: 4
valid_bs: 4
workers: 1

progress_bar_refresh_rate: 1

pseudo_train: 0

save_topk: 1
fold: 5

task:
    #kind: 'detect'
    #kind: 'classify'
    kind: 'depth'
    #kind: 'depth2'
    #condition: 'nfn'
    condition: 'scs'
    #condition: 'ss'

image_size: 384
early_stop:
    monitor: "val_loss"
    mode: "min"
    patience: 999
    verbose: 1

trainer:
    max_epochs: 17
    min_epochs: 17
    enable_progress_bar: True
    precision: "16-mixed"
    devices: 1

model:
    name: "eff"
    loss_smooth: 0.0
    optimizer_params:
        lr: 0.001
        #lr: 0.00007
        weight_decay: 0.0001
    scheduler:
        #name: "CosineAnnealingLR"
        name: "cosine_with_warmup"
        params:
            CosineAnnealingLR:
                T_max: 20
                eta_min: 1.0e-5
                last_epoch: -1
            cosine_with_warmup:
                num_training_steps: 17
                num_warmup_steps: 2
                num_cycles: 0.5
                last_epoch: -1


Overwriting config.yaml


In [ ]:
with open("config.yaml", "r") as file_obj:
    config = yaml.safe_load(file_obj)

In [ ]:
class DepthDetectDataset(Dataset):
    def __init__(self, coor, meta, condition, usage='train'):
        if condition == 'scs':
            coor = coor.loc[coor.condition=='Spinal Canal Stenosis']
        elif condition == 'ss':
            coor = coor.loc[(coor.condition=='Left Subarticular Stenosis') | (coor.condition=='Right Subarticular Stenosis')]
        elif condition == 'nfn':
            coor = coor.loc[(coor.condition=='Right Neural Foraminal Narrowing') | (coor.condition=='Left Neural Foraminal Narrowing')]
        g_coor = coor.groupby('study_id').count()
        if condition == 'scs':
            self.id = g_coor.loc[g_coor.series_id==5].reset_index().study_id.unique()
        else:
            self.id = g_coor.loc[g_coor.series_id==10].reset_index().study_id.unique()
        self.coor = coor
        self.meta = meta
        self.condition = condition
        self.usage = usage
        self.id = list(set(self.id) - set([3637444890]))
        #self.id = [2773343225]
        #self.id = [1782095928]
        if condition == 'ss':
            self.resize = v2.Resize((256, 256))
        else:
            self.resize = v2.Resize((384, 384))
        self.rec_transforms = A.Compose([
            A.ShiftScaleRotate(shift_limit=0.2, scale_limit=(-0.2, 0.2), rotate_limit=15, border_mode=0, p=1),
            A.Resize(384, 384),
        ])
    def __getitem__(self, index):
        study_id = self.id[index]
        if self.condition == 'scs':
            volume, label = self.for_scs(study_id)
        elif self.condition == 'nfn':
            volume, label = self.for_nfn(study_id)
        if self.condition == 'ss':
            volume, label = self.for_ss(study_id)
        return volume, label, torch.tensor([study_id])

    def for_scs(self, study_id):
        depth = 32
        meta = self.meta.loc[(self.meta.study_id==study_id) & (self.meta.series_description=='Sagittal T2/STIR')]
        meta = meta.sort_values('ipp_x', ascending=True).reset_index(drop=True)
        img = [self.load_dicom(f'/content/train_images/{row.study_id}/{row.series_id}/{row.instance_number}.dcm') for _, row in meta.iterrows()]
        coor = self.coor.loc[(self.coor.study_id==study_id) & (self.coor.condition=='Spinal Canal Stenosis')]
        coor_dict = {}
        for _, row in coor.iterrows():
            series_id, instance_number = row.series_id, row.instance_number
            target_row = meta.loc[(meta.series_id==series_id) & (meta.instance_number==instance_number)]
            idx = target_row.index[0]
            height, width = img[idx].shape
            z = idx if len(img) < depth else round((idx/len(img))*depth)
            coor_dict[row.level] = torch.tensor([z]).to(torch.long)
        volume = self.normalize(torch.cat([self.resize(torch.tensor(i)[None, ...]).to(torch.float32) for i in img]).contiguous())
        if volume.shape[0] < depth:
            volume = torch.cat([volume, torch.zeros(depth-volume.shape[0], volume.shape[1], volume.shape[2])])
        elif volume.shape[0] > depth:
            volume = torch.nn.functional.interpolate(volume[None, None, ...], (depth, volume.shape[1], volume.shape[2])).squeeze()
        #if self.usage == 'train':
        #    volume = torch.tensor(self.rec_transforms(image=volume.numpy().transpose((1, 2, 0)))['image'].transpose((2, 0, 1))).to(torch.float32)
        return volume, coor_dict

    def for_nfn(self, study_id):
        depth = 32
        meta = self.meta.loc[(self.meta.study_id==study_id) & (self.meta.series_description=='Sagittal T1')]
        meta = meta.sort_values('ipp_x', ascending=True).reset_index(drop=True)
        img = [self.load_dicom(f'/content/train_images/{row.study_id}/{row.series_id}/{row.instance_number}.dcm') for _, row in meta.iterrows()]
        coor = self.coor.loc[(self.coor.study_id==study_id)]
        coor_dict = {}
        for _, row in coor.iterrows():
            series_id, instance_number = row.series_id, row.instance_number
            target_row = meta.loc[(meta.series_id==series_id) & (meta.instance_number==instance_number)]
            idx = target_row.index[0]
            height, width = img[idx].shape
            z = idx if len(img) < depth else round((idx/len(img)*depth))
            if row.condition == 'Right Neural Foraminal Narrowing':
                coor_dict['right_' + row.level] = torch.tensor([z]).to(torch.long)
            else:
                coor_dict['left_' + row.level] = torch.tensor([z]).to(torch.long)
        volume = self.normalize(torch.cat([self.resize(torch.tensor(i)[None, ...]).to(torch.float32) for i in img]).contiguous())
        if volume.shape[0] < depth:
            volume = torch.cat([volume, torch.zeros(depth-volume.shape[0], volume.shape[1], volume.shape[2])])
        elif volume.shape[0] > depth:
            volume = torch.nn.functional.interpolate(volume[None, None, ...], (depth, volume.shape[1], volume.shape[2])).squeeze()
        return volume, coor_dict
    """
    def normalize(self, x):
        lower, upper = np.percentile(x, (1, 99))
        x = np.clip(x, lower, upper)
        x = x - np.min(x)
        x = x / np.max(x)
        return x
    """
    def normalize(self, x):
        upper = torch.quantile(x, torch.tensor([0.99]))
        lower = torch.quantile(x, torch.tensor([0.01]))
        x = torch.clip(x, lower, upper)
        x = x - torch.min(x)
        x = x / (torch.max(x)+1e-6)
        return x


    def __len__(self):
        return len(self.id)

    def load_dicom(self, path):
        dicom = dcm.dcmread(path)
        data = dicom.pixel_array
        return data


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class SCSDetectLoss(nn.Module):
    def __init__(self):
        super(SCSDetectLoss, self).__init__()

    def forward(self, outputs, targets):
        loss = 0
        for level in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']:
            _loss = nn.functional.l1_loss(outputs[level], targets[level])
            loss += _loss
        return loss/5
class SCSDetectRelativePositionLoss(nn.Module):
    def __init__(self):
        super(SCSDetectRelativePositionLoss, self).__init__()

    def forward(self, outputs, targets):
        loss = 0
        for level in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']:
            _loss = nn.functional.l1_loss(outputs[level], targets[level])
            loss += _loss
        loss/5
        relative_positional_loss = 0
        for l in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']:
            relative_positional_pred_list = []
            relative_positional_tar_list = []
            for ll in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']:
                if l == ll:
                    continue
                pred1, pred2 = outputs[l], outputs[ll]
                tar1, tar2 = targets[l], targets[ll]
                dist_pred = torch.sqrt((pred1[:, 0]-pred2[:, 0])**2 + (pred1[:, 1]-pred2[:, 1])**2)
                dist_tar = torch.sqrt((tar1[:, 0]-tar2[:, 0])**2 + (tar1[:, 1]-tar2[:, 1])**2)
                relative_positional_pred_list.append(dist_pred)
                relative_positional_tar_list.append(dist_tar)
            relative_positional_pred = torch.cat(relative_positional_pred_list)
            relative_positional_tar = torch.cat(relative_positional_tar_list)
            relative_positional_loss += nn.functional.l1_loss(relative_positional_pred, relative_positional_tar)
        return loss/5 + relative_positional_loss/5
class _SCSDetectLoss(nn.Module):
    def __init__(self):
        super(SCSDetectLoss, self).__init__()
        self.map = {'x': 0, 'y': 1}
    def forward(self, outputs, targets):
        loss = 0
        for level in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']:
            for c in ['x', 'y']:
                _loss = nn.functional.cross_entropy(outputs[level][c], targets[level][:, self.map[c]].to(torch.long))
                loss += _loss
        return loss/10

class NFNSSDetectLoss(nn.Module):
    def __init__(self):
        super(NFNSSDetectLoss, self).__init__()

    def forward(self, outputs, targets):
        loss = 0
        for level in ['left_L1/L2', 'left_L2/L3', 'left_L3/L4', 'left_L4/L5', 'left_L5/S1',
                  'right_L1/L2', 'right_L2/L3', 'right_L3/L4', 'right_L4/L5', 'right_L5/S1']:
            _loss = nn.functional.l1_loss(outputs[level], targets[level])
            loss += _loss
        return loss/10

class SCSDepthDetectLoss(nn.Module):
    def __init__(self):
        super(SCSDepthDetectLoss, self).__init__()

    def forward(self, outputs, targets):
        loss = 0
        for level in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']:
            #print(outputs[level].shape, targets[level].shape)
            _loss = nn.functional.cross_entropy(outputs[level], targets[level].reshape(-1))
            loss += _loss
        return loss/5

class NFNSSDepthDetectLoss(nn.Module):
    def __init__(self):
        super(NFNSSDepthDetectLoss, self).__init__()

    def forward(self, outputs, targets):
        loss = 0
        for level in ['left_L1/L2', 'left_L2/L3', 'left_L3/L4', 'left_L4/L5', 'left_L5/S1',
                  'right_L1/L2', 'right_L2/L3', 'right_L3/L4', 'right_L4/L5', 'right_L5/S1']:
            _loss = nn.functional.cross_entropy(outputs[level], targets[level].reshape(-1))
            loss += _loss
        return loss/10

class SCSNFNSSLoss(nn.Module):
    def __init__(self, is_train=False):
        super(SCSNFNSSLoss, self).__init__()
        self.loss = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))
        self.aux_loss_ax = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))
        self.aux_loss_sagt2 = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))
        self.aux_loss_sagt1 = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))
        self.is_train = is_train
    def forward(self, outputs, targets, ax=None, sagt2=None, sagt1=None):
        targets = targets.reshape(-1)
        #print(outputs, targets, outputs.shape, targets.shape)
        ax_loss = 0
        sagt2_loss = 0
        sagt1_loss = 0
        num_loss = 1
        loss = self.loss(outputs, targets)
        if ax is not None:
            ax_loss = self.aux_loss_ax(ax, targets)
            if self.is_train:
                loss += ax_loss*0.5
                num_loss += 1
        if sagt2 is not None:
            sagt2_loss = self.aux_loss_sagt2(sagt2, targets)
            if self.is_train:
                loss += sagt2_loss*0.5
                num_loss += 1
        if sagt1 is not None:
            sagt1_loss = self.aux_loss_sagt1(sagt1, targets)
            if self.is_train:
                loss += sagt1_loss*0.5
                num_loss += 1
        #print(loss)
        loss = loss/num_loss
        return loss, ax_loss, sagt2_loss, sagt1_loss

In [ ]:
from torchvision.ops import StochasticDepth
from typing import List, Dict
from torch import Tensor
#from torchtune.modules import RotaryPositionalEmbeddings

class ConvNextStem(nn.Sequential):
    def __init__(self, in_features: int, out_features: int):
        super().__init__(
            nn.Conv3d(in_features, out_features, kernel_size=(1, 2, 2), stride=(1, 2, 2)),
            nn.GroupNorm(num_groups=1, num_channels=out_features)
        )

class LayerScaler(nn.Module):
    def __init__(self, init_value: float, dimensions: int):
        super().__init__()
        self.gamma = nn.Parameter(init_value * torch.ones((dimensions)),
                                    requires_grad=True)

    def forward(self, x):
        return self.gamma[None,...,None,None] * x

class BottleNeckBlock(nn.Module):
    def __init__(
        self,
        in_features: int,
        out_features: int,
        expansion: int = 4,
        drop_p: float = .0,
        layer_scaler_init_value: float = 1e-6,
    ):
        super().__init__()
        expanded_features = out_features * expansion
        self.block = nn.Sequential(
            # narrow -> wide (with depth-wise and bigger kernel)
            nn.Conv3d(
                in_features, in_features, kernel_size=(2, 7, 7), padding='same', bias=False, groups=in_features
            ),
            # GroupNorm with num_groups=1 is the same as LayerNorm but works for 2D data
            nn.GroupNorm(num_groups=in_features, num_channels=in_features),
            # wide -> wide
            nn.Conv3d(in_features, expanded_features, kernel_size=1),
            nn.GELU(),
            # wide -> narrow
            nn.Conv3d(expanded_features, out_features, kernel_size=1),
        )
        #self.layer_scaler = LayerScaler(layer_scaler_init_value, out_features)
        #self.drop_path = StochasticDepth(drop_p, mode="batch")


    def forward(self, x: Tensor) -> Tensor:
        res = x
        x = self.block(x)
        #x = self.layer_scaler(x)
        #x = self.drop_path(x)
        x += res
        return x

class ConvNexStage(nn.Sequential):
    def __init__(
        self, in_features: int, out_features: int, depth: int, **kwargs
    ):
        super().__init__(
            # add the downsampler
            nn.Sequential(
                nn.GroupNorm(num_groups=in_features, num_channels=in_features),
                nn.Conv3d(in_features, out_features, kernel_size=(2, 2, 2), stride=(2, 2, 2))
            ),
            *[
                BottleNeckBlock(out_features, out_features, **kwargs)
                for _ in range(depth)
            ],
        )

class ConvNextEncoder(nn.Module):
    def __init__(
        self,
        in_channels: int,
        stem_features: int,
        depths: List[int],
        widths: List[int],
        drop_p: float = .0,
    ):
        super().__init__()
        self.stem = ConvNextStem(in_channels, stem_features)

        in_out_widths = list(zip(widths, widths[1:]))
        # create drop paths probabilities (one for each stage)
        drop_probs = [x.item() for x in torch.linspace(0, drop_p, sum(depths))]

        self.stages = nn.ModuleList(
            [
                ConvNexStage(stem_features, widths[0], depths[0], drop_p=drop_probs[0]),
                *[
                    ConvNexStage(in_features, out_features, depth, drop_p=drop_p)
                    for (in_features, out_features), depth, drop_p in zip(
                        in_out_widths, depths[1:], drop_probs[1:]
                    )
                ],
            ]
        )


    def forward(self, x):
        x = self.stem(x)
        for stage in self.stages:
            x = stage(x)
        return x

class ClassificationHead(nn.Sequential):
    def __init__(self):
        super().__init__(
            nn.AdaptiveAvgPool3d((1, 1, 1)),
            nn.Flatten(1),
            nn.LayerNorm(512),
            nn.Linear(512, 3)
        )
class Flatten(nn.Sequential):
    def __init__(self):
        super().__init__(
            nn.AdaptiveAvgPool3d((1, 1, 1)),
            nn.Flatten(1),
            nn.LayerNorm(512)
        )

class ConvNextNFNDepthDetect(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = ConvNextEncoder(in_channels=1, stem_features=32, depths=[3,3,9,3], widths=[64, 128, 256, 512])
        self.flatten = nn.Sequential(nn.AdaptiveAvgPool3d((1,1,1)),
                                    nn.Flatten(1),
                                    nn.LayerNorm(512))
        self.ll1 = nn.Linear(512, 32)
        self.ll2 = nn.Linear(512, 32)
        self.ll3 = nn.Linear(512, 32)
        self.ll4 = nn.Linear(512, 32)
        self.ll5 = nn.Linear(512, 32)
        self.rl1 = nn.Linear(512, 32)
        self.rl2 = nn.Linear(512, 32)
        self.rl3 = nn.Linear(512, 32)
        self.rl4 = nn.Linear(512, 32)
        self.rl5 = nn.Linear(512, 32)
    def forward(self, x, label=None):
        x = x.unsqueeze(1)
        x = self.encoder(x)
        x = self.flatten(x)
        ll1 = self.ll1(x)
        ll2 = self.ll2(x)
        ll3 = self.ll3(x)
        ll4 = self.ll4(x)
        ll5 = self.ll5(x)
        rl1 = self.rl1(x)
        rl2 = self.rl2(x)
        rl3 = self.rl3(x)
        rl4 = self.rl4(x)
        rl5 = self.rl5(x)
        return {'left_L1/L2': ll1,'left_L2/L3': ll2,'left_L3/L4': ll3, 'left_L4/L5': ll4, 'left_L5/S1': ll5,
                'right_L1/L2': rl1, 'right_L2/L3': rl2, 'right_L3/L4': rl3, 'right_L4/L5': rl4, 'right_L5/S1': rl5}

class ConvNextSCSDepthDetect(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = ConvNextEncoder(in_channels=1, stem_features=64, depths=[3,3,9,3], widths=[128, 256, 512, 1024])
        self.flatten = nn.Sequential(nn.AdaptiveAvgPool3d((1,1,1)),
                                    nn.Flatten(1),
                                    nn.LayerNorm(1024),
                                     )
        self.l1 = nn.Linear(1024, 32)
        self.l2 = nn.Linear(1024, 32)
        self.l3 = nn.Linear(1024, 32)
        self.l4 = nn.Linear(1024, 32)
        self.l5 = nn.Linear(1024, 32)
    def forward(self, x, label=None):
        x = x.unsqueeze(1)
        x = self.encoder(x)
        x = self.flatten(x)
        l1 = self.l1(x)
        l2 = self.l2(x)
        l3 = self.l3(x)
        l4 = self.l4(x)
        l5 = self.l5(x)
        return {'L1/L2': l1, 'L2/L3': l2, 'L3/L4': l3, 'L4/L5': l4, 'L5/S1': l5}

In [ ]:
from traitlets.config.loader import ConfigError
VAL_SCORE = []

class DepthDetectModule(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        if config['task']['condition'] == 'scs':
            #self.model = ConvNextSCSDepthDetect()
            self.model = ConvNextSCSDepthDetect()
            self.loss_module = SCSDepthDetectLoss()
        elif config['task']['condition'] == 'nfn':
            self.model = ConvNextNFNDepthDetect()
            self.loss_module = NFNSSDepthDetectLoss()
        self.val_step_outputs = []
        self.val_step_labels = []
        #self.ema = ExponentialMovingAverage(self.model.parameters(), decay=0.995)
        #self.ema.to(device)

        #self.model = torch.optim.swa_utils.AveragedModel(self.model,
        #                                                 multi_avg_fn=torch.optim.swa_utils.get_ema_multi_avg_fn(0.999))

    def forward(self, batch):
        preds = self.model(batch)
        return preds

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), **self.config['model']["optimizer_params"])
        if self.config['model']["scheduler"]["name"] == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(
                optimizer,
                **self.config['model']["scheduler"]["params"]["CosineAnnealingLR"],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        elif self.config['model']["scheduler"]["name"] == "ReduceLROnPlateau":
            scheduler = ReduceLROnPlateau(
                optimizer,
                **self.config['model']["scheduler"]["params"]["ReduceLROnPlateau"],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
        elif self.config['model']['scheduler']['name'] == 'ChrisLR':
            scheduler = ChrisLR(
                optimizer,
                **self.config['model']["scheduler"]["params"]["ChrisLR"],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
        elif self.config['model']['scheduler']['name'] == 'cosine_with_warmup':
            print('cosine with warmup')
            print(self.config['model']['scheduler']['params']['cosine_with_warmup'])
            scheduler = transformers.get_cosine_schedule_with_warmup(
                optimizer,
                **self.config['model']['scheduler']['params']['cosine_with_warmup'],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        else:
            return {"optimizer": optimizer}

    def training_step(self, batch, batch_idx):
        volume, label, _ = batch
        preds = self.model(volume)
        loss = self.loss_module(preds, label)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, batch_size=config['train_bs'])
        for param_group in self.trainer.optimizers[0].param_groups:
            lr = param_group["lr"]
        self.log("lr", lr, on_step=True, on_epoch=False, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        """Add TTA"""
        volume, label, _ = batch
        preds = self.model.forward(volume)
        loss = self.loss_module(preds, label)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        #self.val_step_outputs.append(preds)
        #self.val_step_labels.append(labels)
        return loss

    def on_validation_epoch_end(self):
        if self.trainer.global_rank == 0:
            print(f"\nEpoch: {self.current_epoch}", flush=True)
        return 0

    def optimizer_step(self, *args, **kwargs):
        super().optimizer_step(*args, **kwargs)
        #self.ema.update()

In [ ]:
train = pd.read_csv('/content/train.csv')
train_dummy = pd.read_csv('/content/train.csv')
train_series = pd.read_csv('/content/train_series_descriptions.csv')
train_coor = pd.read_csv('/content/train_label_coordinates.csv')
train_meta = pd.read_parquet('/content/meta.parquet')

In [ ]:
train_dummy = train_dummy.fillna('Normal/Mild')

In [ ]:
label = train.columns[1:]
kfold = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for i, (train_idx, valid_idx) in enumerate(kfold.split(train_dummy, train_dummy[label])):
    train_dummy.loc[valid_idx, 'fold'] = i
train_series = train_series.merge(train_dummy[['study_id', 'fold']], on='study_id')
train_coor = train_coor.merge(train_dummy[['study_id', 'fold']], on='study_id')
train = train.merge(train_dummy[['study_id', 'fold']], on='study_id')
train_meta = train_meta.merge(train_dummy[['study_id', 'fold']], on='study_id')

In [ ]:
T_MAX = config["model"]["scheduler"]["params"]["CosineAnnealingLR"]["T_max"]
num_training_steps = config["model"]["scheduler"]["params"]["cosine_with_warmup"]["num_training_steps"]
num_warmup_steps = config["model"]["scheduler"]["params"]["cosine_with_warmup"]["num_warmup_steps"]

In [ ]:
%%time
import warnings
warnings.filterwarnings("ignore")
##############DEPTH DETECT#########################
print(config['task']['condition'])
print(config['task']['kind'])
print(T_MAX)
#for i in [5]:
for i in [0, 1, 2, 3, 4]:
    train_df = train.loc[train.fold != i]
    valid_df = train.loc[train.fold == i]
    train_df = train_df.drop('fold', axis=1)
    valid_df = valid_df.drop('fold', axis=1)
    train_series_df = train_series.loc[train_series.fold != i]
    valid_series_df = train_series.loc[train_series.fold == i]
    train_series_df = train_series_df.drop('fold', axis=1)
    valid_series_df = valid_series_df.drop('fold', axis=1)
    train_coor_df = train_coor.loc[train_coor.fold != i]
    valid_coor_df = train_coor.loc[train_coor.fold == i]
    train_coor_df = train_coor_df.drop('fold', axis=1)
    valid_coor_df = valid_coor_df.drop('fold', axis=1)
    train_meta_df = train_meta.loc[train_meta.fold != i]
    valid_meta_df = train_meta.loc[train_meta.fold == i]
    train_meta_df = train_meta_df.drop('fold', axis=1)
    valid_meta_df = valid_meta_df.drop('fold', axis=1)
    dataset_train = DepthDetectDataset(train_coor_df, train_meta_df, config['task']['condition'], 'train')
    dataset_validation = DepthDetectDataset(valid_coor_df, valid_meta_df, config['task']['condition'], 'valid')
    data_loader_train = DataLoader(
        dataset_train,
        batch_size=config["train_bs"],
        shuffle=True,
        num_workers=4,
        pin_memory=True,
    )
    data_loader_validation = DataLoader(
        dataset_validation,
        batch_size=config["valid_bs"],
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    checkpoint_callback = ModelCheckpoint(
        save_weights_only=True,
        monitor="val_loss",
        dirpath=config["output_dir"],
        mode='min',
        filename=f"{config['task']['condition']}_{config['task']['kind']}_1024_ssr_{i}",
        save_top_k=config["save_topk"],
        verbose=1,
    )

    progress_bar_callback = TQDMProgressBar(
        refresh_rate=config["progress_bar_refresh_rate"]
    )

    early_stop_callback = EarlyStopping(**config["early_stop"])
    _c, _k = config['task']['condition'], config['task']['kind']
    wandb_logger = WandbLogger(project=f'rsna_spine_{_c}_{_k}', # group runs in "MNIST" project
                            log_model='all') # log all new checkpoints during training

    trainer = pl.Trainer(
        logger=wandb_logger,
        callbacks=[checkpoint_callback, early_stop_callback, progress_bar_callback],
        **config["trainer"],
    )

    config["model"]["scheduler"]["params"]["CosineAnnealingLR"]["T_max"] = T_MAX*len(data_loader_train)/config["trainer"]["devices"]
    config["model"]["scheduler"]["params"]["cosine_with_warmup"]["num_training_steps"] = int(num_training_steps*len(data_loader_train)/config["trainer"]["devices"])
    config["model"]["scheduler"]["params"]["cosine_with_warmup"]["num_warmup_steps"] = int(num_warmup_steps*len(data_loader_train)/config["trainer"]["devices"])
    model = DepthDetectModule(config=config)
    trainer.fit(model, data_loader_train, data_loader_validation)
    wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


scs
depth
20


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: syujinko. Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type                   | Params | Mode 
---------------------------------------------------------------
0 | model       | ConvNextSCSDepthDetect | 52.7 M | train
1 | loss_module | SCSDepthDetectLoss     | 0      | train
---------------------------------------------------------------
52.7 M    Trainable params
0         Non-trainable params
52.7 M    Total params
210.698   Total estimated model params size (MB)
158       Modules in train mode
0         Modules in eval mode


cosine with warmup
{'num_training_steps': 6443, 'num_warmup_steps': 758, 'num_cycles': 0.5, 'last_epoch': -1}


...


Epoch: 2


INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved by 0.038 >= min_delta = 0.0. New best score: 1.049
INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 1137: 'val_loss' reached 1.04858 (best 1.04858), saving model to '/content/gdrive/MyDrive/RSNA_SPINE/models/scs_depth_1024_ssr_0-v4.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]


Epoch: 3


INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved by 0.124 >= min_delta = 0.0. New best score: 0.925
INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 1516: 'val_loss' reached 0.92499 (best 0.92499), saving model to '/content/gdrive/MyDrive/RSNA_SPINE/models/scs_depth_1024_ssr_0-v4.ckpt' as top 1
